In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
import traci
import sys
import argparse
import tensorflow as tf
import numpy as np
from dotenv import load_dotenv

# Import internal modules
from rl_package.rl_logic.Environnement import EnvironnementSumo
from rl_package.rl_logic.Agent import AgentSumo
from rl_package.params import *


In [21]:


# Load environment variables
load_dotenv()


#SUMO_BIN = r"C:/Program Files/rl_project/Eclipse/Sumo/bin/sumo.exe"
SIMUL_CONFIG = r"double_traffic/double_traffic.sumo.cfg"


WINDOW=2000
BATCH_SIZE=6
# SUMO command


In [22]:

def preprocess():
    """
    Determines the number of inputs and outputs required for the model.
    """
    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']
    env = EnvironnementSumo(sumoCmd, WINDOW)
    inputs_per_agents = []
    outputs_per_agents = []
    for trafficlight in env.trafficlights_ids:

    # Get the number of lanes that are not intersections
        n_lanes = len(env.control_lanes(trafficlight))
        inputs_per_agents.append(n_lanes*2)

        # Get the number of valid traffic light phases (excluding yellow phases)
        n_outputs = len(env.get_phase_without_yellow(trafficlight)[0])
        outputs_per_agents.append(n_outputs)

    # Get the number of agents

    env.close()
    return inputs_per_agents, outputs_per_agents  # Inputs: lane states (queue + vehicle count), Outputs: traffic light phases


In [23]:

def train_models(inputs_per_agents, outputs_per_agents, type_model="DQN"):
    """
    Trains a reinforcement learning model to optimize traffic lights.
    Saves the trained model after completion.
    """

    agents = [AgentSumo(type_model, inputs, outputs) for inputs,outputs  in zip(inputs_per_agents,outputs_per_agents)]
    for agent in agents:
        agent.build_model()
        # model_path = f"models/{type_model}.keras"
        # if os.path.exists(model_path):
        #     print(f"🔄 Loading pre-trained model {type_model}...")
        #     agents[i].model_action=tf.keras.models.load_model(model_path)
        #     agents[i].model_target=tf.keras.models.load_model(model_path)
        #print('fvvgfv', agent.n_inputs, agent.n_outputs)
    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']

    for episode in range(EPISODE):
        print(f'🔄 Episode {episode}/{EPISODE}')
        env = EnvironnementSumo(sumoCmd, WINDOW)
        #epsilon = max(1 - episode / EPISODE, 0.01)  # Decaying epsilon for exploration
        epsilon=0.2
        # récupère le nom des agents
        traffic_lights = env.trafficlights_ids



        # appel une fonction et récupère une partie du dictionnaire
        states = [env.get_states_per_traffic_light(traffic_light) for traffic_light in traffic_lights]

        for _ in range(50):  # Steps per episode
            actions = [agent.epsilon_greedy_policy(np.array(states[i]),epsilon) for i, agent in enumerate(agents)]
            #print('ici')
            next_states, rewards = env.step(actions)
            for i in range(len(agents)):
                agents[i].add_to_memory(np.array(states[i]), np.array(actions[i]), np.array(rewards[i]), np.array(next_states[i]))
            states = next_states
            # Train the model if there is enough experience in memory

            if len(agents[0].replay_buffer) >= BATCH_SIZE * 1:
                for agent in agents:
                    agent.training_step(BATCH_SIZE)

            # Stop the simulation if there are no vehicles left
            if env.get_total_number_vehicles() == 0:
                break

        # Update target network every 5 episodes for Double/Dueling DQN
        if episode % 5 == 0 and type_model != 'DQN':
            for agent in agents:
                agent.model_target.set_weights(agent.model_action.get_weights())

        env.close()
    return agents

    # Save the trained model
    # model_path = f"models/{type_model}.keras"
    # agents[i].model_action.save(model_path)
    # print(f"✅ Model saved at: {model_path}")


In [24]:

def scenario(agents):
    """
    Runs a SUMO simulation using the trained agent.
    """
    sumoCmd = [SUMO_GUI_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']
    env = EnvironnementSumo(sumoCmd, WINDOW)
    env.full_simul(agents)


In [25]:

type_model = "2DQN"
inputs_per_agents, outputs_per_agents = preprocess()
agents  = train_models(inputs_per_agents, outputs_per_agents,type_model)


 Retrying in 1 seconds


Error: Could not access configuration 'double_traffic/double_traffic.sumo.cfg'.
Quitting (on error).
/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/traci/main.py:154: UserWarning: Could not connect to TraCI server using port 57671 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +
Error: Could not access configuration 'double_traffic/double_traffic.sumo.cfg'.
Quitting (on error).


 Retrying in 1 seconds


/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/traci/main.py:154: UserWarning: Could not connect to TraCI server using port 57676 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +
Error: Could not access configuration 'double_traffic/double_traffic.sumo.cfg'.
Quitting (on error).


 Retrying in 1 seconds


...

/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/traci/main.py:154: UserWarning: Could not connect to TraCI server using port 57693 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +
Error: Could not access configuration 'double_traffic/double_traffic.sumo.cfg'.
Quitting (on error).


 Retrying in 1 seconds


/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/traci/main.py:154: UserWarning: Could not connect to TraCI server using port 57698 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +
Error: Could not access configuration 'double_traffic/double_traffic.sumo.cfg'.
Quitting (on error).


 Retrying in 1 seconds


KeyboardInterrupt: 

In [ ]:
scenario(agents)

 Retrying in 1 seconds


FatalTraCIError: Connection closed by SUMO.